### 6.1网易新闻中心爬虫

In [ ]:
#爬取网页的html源码
import urllib.request  
url = 'http://news.163.com'  
head = {}  
head['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' 
req = urllib.request.Request(url, headers=head)
response = urllib.request.urlopen(req)
h1 = response.read()
type(h1)       #显示变量h1的数据类型

In [2]:
#将python2的默认编码格式设置为utf-8
import sys  
sys.getdefaultencoding()   #返回程序文件的默认编码格式

'utf-8'

In [4]:
#二进制类型与字符串类型的转换
s='你们好 you are welcome'   #python3中字符串的默认编码为utf-8
type(s)                      #变量s的类型为str
a=s.encode('gb2312')         #将字符串按gb2312的编码格式编码为字节对象a 
type(a)                      #返回a的类型为bytes
print(a)                     #输出字节对象a
a.decode('utf-8')            #将字节对象以utf-8格式解码为字符串，程序报错

h2=h1.decode('gbk')
print(h2)

b'\xc4\xe3\xc3\xc7\xba\xc3 you are welcome'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 0: invalid continuation byte

In [ ]:
#提取URL与新闻标题
import re
content=re.findall('a href="(http://.+?.html)">(.+?)</a>',h2)
print(content)     #输出列表内容

### 6.2 通过session模拟登陆豆瓣

In [ ]:
#第一步 定义获取验证码图片，并输入验证码信息的函数get_idcode()
import re,requests,time
def get_idcode(session,login_url,head,data):
    try:
        htmls = session.get(login_url,headers=head).text
        # 获取验证码图片地址和验证码id
        captcha_img=re.findall(img_pattern, htmls)[0]
        captcha_id = re.findall(id_pattern, htmls)[0]
        # 将验证码图片保存到本地文件
        with open('idcode.jpg', 'wb') as f:
            captcha_img = session.get(captcha_img) #根据验证码图片地址获取验证码图片
            f.write(captcha_img.content)
        # 等待用户输入验证码
        captcha = input('输入验证码')
        # 将验证码和id放在登陆参数上面
        data['captcha-solution'] = captcha
        data['captcha-id'] = captcha_id
    except Exception as e:
        print(e,'不需要输入验证码')

In [ ]:
#第二步 定义登陆函数login()
def login(session,login_url,head,post_data):
    # 登陆网站，用同一个会话信息，这个post_data就是登陆的参数
    session.post(login_url,headers=head,data=post_data)

In [ ]:
#第三步 编写爬取影评数据的功能函数
def get_html(session,target):
    response=session.get(target, headers=head)
    response.encoding="utf-8"
    return response.text

def get_data(html):
    comment=re.findall(comment_pattern,html)  #取评论
    #score=re.findall('<span class="allstar(.+?) rating"',html)  #取评分
    #user=re.findall('<a href="https://www.douban.com/people/(.+?)/"',html) 
    next_page=re.findall(next_pattern,html) 
    return comment,next_page

def sort_data(info,f):
    #user=info[0]
    #score=info1]
    comment=info[0]
    for n in range(len(comment)):
        f.write(comment[n].strip()+'\n')

In [ ]:
#第四步 编写入口程序
f = open('d:/douban.txt','a',encoding='utf-8')
base_url = 'https://movie.douban.com/subject/26752852/comments' 
first_url='https://movie.douban.com/subject/26752852/comments?status=P'
img_pattern = '<img id="captcha_image" src="([\d\D]*?)" alt="captcha" class="captcha_image"/>'
id_pattern = '<input type="hidden" name="captcha-id" value="([\d\D]*?)"/>'
comment_pattern='<p class="">([\d\D]*?)</p>'
next_pattern='<a href="(.+?)" data-page="" class="next">后页 ></a>'
head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}
login_url='https://accounts.douban.com/login'
data = {'source': 'None',
    'redir': 'https://www.douban.com/',
    'form_email': 'lizhouping_2006@126.com',
    'form_password': 'lzp54775477',
    'login': '登录'}
session = requests.Session()
get_idcode(session,login_url,head,data)
login(session,login_url,head,data)
target=first_url
i=0
while True:
    html=get_html(session,target)
    info=get_data(html)
    sort_data(info,f)
    print(str(i)+' 页下载完毕')
    if info[1]:
        i=i+1
        target = base_url + info[1][0]
        html=get_html(session,target)
        info=get_data(html)
        sort_data(info,f)
        time.sleep(1)
    else:
        print('所有影评下载完毕')
        break
f.close()


### 6.3 链家二手房信息爬虫

In [ ]:
#第一步 编写提取二手房详情页网址的函数
import urllib.request,re 
from bs4 import BeautifulSoup
def abs_apart_link(html,key_list):
    soup=BeautifulSoup(html,'html.parser')
#在类名为title的<div>标签的直接子节点中提取<a>标签
    apart_list=soup.select('div.title > a') 
    for apart in apart_list:
        key_list.append(apart.attrs['href'])  #提取Tag对象的href属性值

In [ ]:
#第二步 编写提取二手房详情页面信息的函数
def abs_apart_info(url):
    response = urllib.request.urlopen(url,timeout=3)
    html =response.read() 
    soup=BeautifulSoup(html,'html.parser')
    #利用正则表达式提取房屋编号
    num=re.findall('ershoufang/(.+?).html',url)
    f1.write(num[0]+',')
    #提取总价标签
    total=soup.find_all('span',class_='total')[0].text.strip()
    f1.write(total+',')
    #提取小区标签
    f1.write(soup.select('div.communityName a.info')[0].text.strip()+',') 
    #提取区域标签
    a=soup.select('div.areaName span.info')[0].text.strip()
    f1.write(a.replace('\xa0',' ')+',')
    #提取基本属性和交易属性标签
    for apart in soup.select('li span.label'):      
        f1.write(apart.next_sibling.strip()+',')
    f1.write('\n')
    f1.flush()

In [ ]:
#第三步 遍历并提取100个二手房列表页面中所有二手房源详情页地址
key_list=[]
url = 'http://sh.lianjia.com/ershoufang/pg'
for i in range(100):
    print('第'+str(i+1)+'页')
    pageurl=url+str(i+1)
    try:
        response = urllib.request.urlopen(pageurl,timeout=5)
        html =response.read()
    except:
        print("爬取失败_____")
        continue
    print("爬取成功_____")
    abs_apart_key(html,key_list)

In [ ]:
#第四步提取详情页信息，并写入文本文件中
Import re
#写入字段名
f1 = open('d:/lianjia.txt','a')
url0=key_list[0]
response = urllib.request.urlopen(url0)
html =response.read() 
soup=BeautifulSoup(html,'html.parser')
f1.write('房屋编号'+','+'总价'+','+'小区'+','+'区域')
#提取基本属性和交易属性的字段名，并写入文件
for apart in soup.select('li span.label'):
    f1.write(','+apart.text.strip())
f1.write('\n')
#写入详情页信息
for i in range(len(key_list)):
    try:
        abs_apart_info(key_list[i])
        print(str(i)+' 提取成功')
    except:
        print(str(i)+' 提取失败')
        continue
f1.close()

### 6.4 爬取拉勾网json格式数据

In [ ]:
#第一步 定义getPageNumber函数，该函数负责返回搜索页面的总页数
import requests,re
import time
from bs4 import BeautifulSoup
import xlwt as ExcelWrite
def getPageNumber(url,head,keyword):  
    data = {
    'first':True,
    'pn':1,
    'kd':keyword}
    page = requests.post(url=url,headers=head,data=data)
    page.encoding = 'utf-8'
    page_json = page.json()
    pageNumbe = int(page_json['content']['positionResult']['totalCount']/15)
    return pageNumbe 

In [ ]:
#第二步 定义get_page函数，返回指定搜索页的职位基本信息
def get_page(pn,url,head,keyword):
    if pn == 1:
        first = True
    else:
        first = False
    data = {
    'first':first,
    'pn':pn,
    'kd':keyword}
    page = requests.post(url=url,headers=head,data=data,timeout=5)
    page.encoding = 'utf-8'
    page_json = page.json()
    position = page_json['content']['positionResult']['result']
    return position

In [ ]:
#第三步 定义get_description函数，返回职位的详细描述信息
def get_description(jobID,head):
    job_url='https://www.lagou.com/jobs/'+str(jobID)+'.html'
    page=requests.get(job_url,headers=head,timeout=5)
    page.encoding = 'utf-8'
    soup=BeautifulSoup(page.text,'html.parser')
    job_description = soup.select('dd[class="job_bt"]')
    job_description = str(job_description[0])
    rule = re.compile(r'<[^>]+>')
    result = rule.sub('', job_description)
    return result

In [ ]:
#第四步 定义save_excel函数，将每条职位信息保存到excel文件中
def save_excel(i0,job,sheet,tags,head):
    for j in range(len(tags)-1):         
        sheet.write(i0,j,str(job[tags[j]]))
    try:
        des=get_description(job['positionId'],head)
        print(str(job['positionId'])+ '职位描述抓取成功')        
    except:
        print(str(job['positionId'])+ '职位描述抓取失败')
        des=''
    sheet.write(i0,len(tags)-1,des)

In [ ]:
#第五步 编写入口程序
url='https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false&isSchoolJob=0'  #搜索页面的URL地址
key='数据'                     #定义搜索的关键字
head1={省略......}
head2={省略......}
pagenumber=getPageNumber(url,head1,key)
tags=['positionId','positionName','city', 'createTime','companyFullName','education',
'industryField','companySize','firstType','secondType','salary',
'workYear','description']
book = ExcelWrite.Workbook(encoding='utf-8')     #存放职位情况的excel表格
sheet = book.add_sheet('sheet')
i0=0
for j in range(len(tags)):     #向工作表写入表头
    sheet.write(i0,j,str(tags[j]))
for i in range(pagenumber):    #将每个职位信息写入工作表
    try:
        jobs=get_page(i+1,url,head1,key)
        print('第'+str(i+1)+'爬取成功')
        for job in jobs:
            i0=i0+1
            save_excel(i0,job,sheet,tags,head2)
    except:
        print('第'+str(i+1)+'爬取失败') 
book.save("f:\\lagou.xls" )    #保存excel文件